<a href="https://colab.research.google.com/github/gc2321/3235-Machine-Learning/blob/main/assign_2_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Flowers using Transfer Learning in Keras

Assignment 2 - Convolutional Neural Nets and Transfer Learning

By Gordon Chan (qq525548)

In [4]:
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
random.seed(42)

## Loading Data

In [6]:
import tensorflow_datasets as tfds

# Load the VOC 2007 dataset
dataset, info = tfds.load('voc/2007', with_info=True)

# Get the total number of examples
total_examples = info.splits['train'].num_examples

# Calculate the sizes for each split
train_size = int(0.8 * total_examples)
test_size = total_examples - train_size
val_size = int(0.1 * train_size)  # 10% of the training set for validation

# Define the splits
train_split = f'train[:{train_size}]'
test_split = f'train[{train_size}:]'
val_split = f'train[:{val_size}]'

# Load the datasets with the defined splits
train_data = tfds.load('voc/2007', split=train_split)
test_data = tfds.load('voc/2007', split=test_split)
val_data = tfds.load('voc/2007', split=val_split)

# Verify the splits
train_count = 0
for _ in train_data:
    train_count += 1

test_count = 0
for _ in test_data:
    test_count += 1

val_count = 0
for _ in val_data:
    val_count += 1

print(f'Train examples: {train_count}')
print(f'Test examples: {test_count}')
print(f'Validation examples: {val_count}')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/4952 [00:00<?, ? examples/s]

...

Generating train examples...:   0%|          | 0/2501 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/voc/2007/incomplete.G8F1CB_4.0.0/voc-train.tfrecord*...:   0%|          | …

Generating validation examples...:   0%|          | 0/2510 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/voc/2007/incomplete.G8F1CB_4.0.0/voc-validation.tfrecord*...:   0%|       …

Dataset voc downloaded and prepared to /root/tensorflow_datasets/voc/2007/4.0.0. Subsequent calls will reuse this data.
Train examples: 2000
Test examples: 501
Validation examples: 200


In [8]:
info

tfds.core.DatasetInfo(
    name='voc',
    full_name='voc/2007/4.0.0',
    description="""
    This dataset contains the data from the PASCAL Visual Object Classes Challenge,
    corresponding to the Classification and Detection competitions.
    
    In the Classification competition, the goal is to predict the set of labels
    contained in the image, while in the Detection competition the goal is to
    predict the bounding box and label of each individual object.
    annotations.
    """,
    config_description="""
    This dataset contains the data from the PASCAL Visual Object Classes Challenge
    2007, a.k.a. VOC2007.
    
    A total of 9963 images are included in this dataset, where each image
    contains a set of objects, out of 20 different classes, making a total of
    24640 annotated objects.
    
    """,
    homepage='http://host.robots.ox.ac.uk/pascal/VOC/voc2007/',
    data_dir=PosixGPath('/tmp/tmprd9h10zrtfds'),
    file_format=tfrecord,
    download_size=868.85 Mi

In [12]:
class_names = info.features["labels"].names
print(class_names)
#n_classes = info.features["label"].num_classes
print(class_names)
#print(n_classes)

['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
